# PIT on VCT data revisited nonlinear model

# Purpose
PIT on VCT has previously been conducted in:
* [04.03_PIT_VCT_linear_VMM.ipynb](04.03_PIT_VCT_linear_VMM.ipynb)
* [04.03_PIT_VCT_nonlinear_VMM.ipynb](04.03_PIT_VCT_nonlinear_VMM.ipynbb)
...but that was a while a ago, so this is a retake on this one...

# Methodology
* Load VCT data 
* Convert it to prime system
* Regress fx, fy, mz
* simulate with these coefficients

# Setup

In [ ]:
# %load imports.py
## Local packages:

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

## External packages:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
np.set_printoptions(linewidth=150)

import os
import matplotlib.pyplot as plt
#if os.name == 'nt':
#    plt.style.use('presentation.mplstyle')  # Windows

import plotly.express as px 
import plotly.graph_objects as go

import seaborn as sns
import sympy as sp
from sympy.physics.mechanics import (dynamicsymbols, ReferenceFrame,
                                      Particle, Point)
from sympy.physics.vector.printing import vpprint, vlatex
from IPython.display import display, Math, Latex
from vessel_manoeuvring_models.substitute_dynamic_symbols import run, lambdify

import pyro

import sklearn
import pykalman
from statsmodels.sandbox.regression.predstd import wls_prediction_std
import statsmodels.api as sm

from scipy.integrate import solve_ivp

## Local packages:
from vessel_manoeuvring_models.data import mdl

from vessel_manoeuvring_models.symbols import *
from vessel_manoeuvring_models.parameters import *
import vessel_manoeuvring_models.symbols as symbols
from vessel_manoeuvring_models import prime_system
from vessel_manoeuvring_models.models import regression
from vessel_manoeuvring_models.visualization.regression import show_pred, show_pred_captive
from vessel_manoeuvring_models.visualization.plot import track_plot, captive_plot

## Load models:
# (Uncomment these for faster loading):
import vessel_manoeuvring_models.models.vmm_abkowitz as vmm 
from vessel_manoeuvring_models.models.vmm import ModelSimulator

In [ ]:
from vessel_manoeuvring_models.data.wpcc import ship_parameters, df_parameters, ps, ship_parameters_prime, ps_ship

## Load VCT data

In [ ]:
df_VCT_all = pd.read_csv('../data/external/vct.csv', index_col=0)
df_VCT = df_VCT_all.groupby(by=['model_name']).get_group('V2_5_MDL_modelScale')

In [ ]:
df_VCT.describe()

# Subtract the resistance

In [ ]:
df_resistance = df_VCT.groupby(by='test type').get_group('resistance')

X = df_resistance[['u','fx']].copy()
X['u**2'] = X['u']**2
y = X.pop('fx')

model_resistance = sm.OLS(y,X)
results_resistance = model_resistance.fit()

X_pred = pd.DataFrame()
X_pred['u'] = np.linspace(X['u'].min(), X['u'].max(), 20)
X_pred['u**2'] = X_pred['u']**2
X_pred['fx'] = results_resistance.predict(X_pred)

fig,ax=plt.subplots()
df_resistance.plot(x='u', y='fx', style='.', ax=ax)
X_pred.plot(x='u', y='fx', style='--', ax=ax);


In [ ]:
df_VCT_0_resistance = df_VCT.copy()
df_VCT_0_resistance['u**2'] = df_VCT_0_resistance['u']**2
df_VCT_0_resistance['fx']-= results_resistance.predict(df_VCT_0_resistance[['u','u**2']])

## VCT to prime system

In [ ]:
interesting = [
    'u',
    'v',
    'r',
    'delta',
    'fx',
    'fy',
    'mz',
    'thrust',
]
#df_VCT_prime = ps_ship.prime(df_VCT[interesting], U=df_VCT['V'])
df_VCT_prime = ps_ship.prime(df_VCT_0_resistance[interesting], U=df_VCT_0_resistance['V'])

## Regression

### N

In [ ]:
Math(vlatex(vmm.N_qs_eq))

In [ ]:
N_ = sp.symbols('N_')
diff_eq_N = regression.DiffEqToMatrix(ode=vmm.N_qs_eq.subs(N_qs,N_), 
                                      label=N_, base_features=[delta,u,v,r])

X = diff_eq_N.calculate_features(data=df_VCT_prime)
y = diff_eq_N.calculate_label(y=df_VCT_prime['mz'])

model_N = sm.OLS(y,X)
results_N = model_N.fit()

show_pred_captive(X=X,y=y,results=results_N, label=r'$N$')

## Y

In [ ]:
Math(vlatex(vmm.Y_qs_eq))

In [ ]:
Y_ = sp.symbols('Y_')
diff_eq_Y = regression.DiffEqToMatrix(ode=vmm.Y_qs_eq.subs(Y_qs,Y_), 
                                      label=Y_, base_features=[delta,u,v,r])

X = diff_eq_Y.calculate_features(data=df_VCT_prime)
y = diff_eq_Y.calculate_label(y=df_VCT_prime['fy'])

model_Y = sm.OLS(y,X)
results_Y = model_Y.fit()

show_pred_captive(X=X,y=y,results=results_Y, label=r'$Y$')

## X

In [ ]:
Math(vlatex(vmm.X_qs_eq))

In [ ]:
X_ = sp.symbols('X_')
ode=vmm.X_qs_eq.subs([(X_qs,X_),
                      (thrust,0),  # Removing thrust from the equation
                     ])

diff_eq_X = regression.DiffEqToMatrix(ode=ode, 
                                      label=X_, base_features=[delta,u,v,r])

X = diff_eq_X.calculate_features(data=df_VCT_prime)
y = diff_eq_X.calculate_label(y=df_VCT_prime['fx'])

model_X = sm.OLS(y,X)
results_X = model_X.fit()

show_pred_captive(X=X,y=y,results=results_X, label=r'$X$')

In [ ]:
results_summary_X = regression.results_summary_to_dataframe(results_X)
results_summary_Y = regression.results_summary_to_dataframe(results_Y)
results_summary_N = regression.results_summary_to_dataframe(results_N)

In [ ]:
df_parameters_all = df_parameters.copy()
for other in [results_summary_X, results_summary_Y, results_summary_N]:
    df_parameters_all = df_parameters_all.combine_first(other)

df_parameters_all.rename(columns={'coeff':'regressed'}, inplace=True)
df_parameters_all.drop(columns=['brix_lambda'], inplace=True)

df_parameters_all['regressed'] = df_parameters_all['regressed'].combine_first(df_parameters_all['prime'])  # prefer regressed
#df_parameters_all['regressed'].fillna(0,inplace=True)

## Create a simulator with regressed parameters

In [ ]:
model = ModelSimulator(simulator=vmm.simulator, 
                       parameters=df_parameters_all['regressed'], 
                       ship_parameters=ship_parameters, 
                       control_keys=['delta'], 
                       primed_parameters=True,
                       prime_system=ps,)
model.save('../models/model_VCT_abkowitz.pkl')

In [ ]:
outputs = model.forces(inputs = df_VCT_prime)
df_captive = pd.merge(left=outputs, right=df_VCT_prime, 
                      how='left', 
                      left_index=True, 
                      right_index=True,
                      suffixes = ('','_vct'))

df_captive['test type'] = df_VCT['test type']

In [ ]:
captive_plot(df_captive=df_captive, suffixes=['_vct'])

# Simulation

A simulation with the regressed hydrodynamic coefficients can now be conducted. The figures below compare results from these simulation and the original data.

## Load test

In [ ]:
def load_test(id):

    df, units, meta_data = mdl.load(id=id, dir_path='../data/processed/kalman_cut/')
    df.index = df.index.total_seconds()
    df = df.iloc[0:-100].copy()
    df.index-=df.index[0]
    df.sort_index(inplace=True)
    df['thrust'] = df['Prop/PS/Thrust'] + df['Prop/SB/Thrust']
    
    return df



tests = {
    22774 : load_test(id=22774).loc[80:],
    22773 : load_test(id=22773),
    22770 : load_test(id=22770).iloc[100:],
}
    


In [ ]:
results = {}
for id, df_ in tests.items():

    result_regression = model.simulate(df_=df_, 
                                       name='regressed', method='RK45')
    
    results[id] = result_regression


In [ ]:
for id, result_regression in results.items():

    result_regression.track_plot(compare=True);
    
    fig = result_regression.plot(compare=True, ax=ax);
    fig.set_size_inches(15,10)
    plt.tight_layout()

In [ ]:
tests[22774].plot(y='thrust', style='.', alpha=0.02)

fig,ax=plt.subplots()
tests[22774].plot(y=['Prop/PS/Thrust','Prop/SB/Thrust'], style='.', alpha=0.02, ax=ax)
ax.legend()

fig,ax=plt.subplots()
tests[22774].plot(y=['Prop/PS/Rpm','Prop/SB/Rpm'], style='.', alpha=0.02, ax=ax)
ax.legend()

tests[22774].plot(y='delta')


In [ ]:
tests[22774].columns